# Generalized LASSO

This notebook contains examples of how to solve total variation denoising problem.

Reference: Boyd, Stephen, Neal Parikh, and Eric Chu. Distributed optimization and statistical learning via the alternating direction method of multipliers. Now Publishers Inc, 2011.


## Problem Definition

$$\min \frac{1}{2} ||Ax-b||_2^2+\lambda||Fx||_1,$$
where $A$ is an indentity matrix and $F$ is a the difference matrix, in which case the above form reduces to
$$\min_x \frac{1}{2}||x-b||_2^2+\lambda\sum_{i=1}^{n-1}|x_{i+1}-x_i|$$

## Import all necessary modules and add PyGRANSO src folder to system path.

In [1]:
import time
import torch
import sys
## Adding PyGRANSO directories. Should be modified by user
sys.path.append('/home/buyun/Documents/GitHub/PyGRANSO')
from pygranso import pygranso
from pygransoStruct import Options, Data, GeneralStruct 

## Specify torch device, and generate data

In [2]:
device = torch.device( 'cpu')
n = 80
eta = 0.5 # parameter for penalty term
torch.manual_seed(1)
b = torch.rand(n,1)
pos_one = torch.ones(n-1)
neg_one = -torch.ones(n-1)
F = torch.zeros(n-1,n)
F[:,0:n-1] += torch.diag(neg_one,0) 
F[:,1:n] += torch.diag(pos_one,0)
F = F.to(device=device, dtype=torch.double)  # double precision requireed in torch operations 
b = b.to(device=device, dtype=torch.double)

## Spceify optimization variables and corresponding objective and constrained function.

Note: please strictly follow the format of evalObjFunction and combinedFunction, which will be used in the PyGRANSO main algortihm. *X_struct* and *data_in* are always required.

In [3]:
# variables and corresponding dimensions.
var_in = {"x": [n,1]}

def evalObjFunction(X_struct):
    x = X_struct.x
    x.requires_grad_(True)
    
    # objective function
    f = (x-b).t() @ (x-b)  + eta * torch.norm( F@x, p = 1)
    return f

def combinedFunction(X_struct):
    x = X_struct.x
    x.requires_grad_(True)
    
    # objective function
    f = (x-b).t() @ (x-b)  + eta * torch.norm( F@x, p = 1)
    
    # inequality constraint, matrix form
    ci = None
    # equality constraint 
    ce = None

    return [f,ci,ce]

obj_eval_fn = lambda X_struct: evalObjFunction(X_struct)
comb_fn = lambda X_struct: combinedFunction(X_struct)

## Specify user-defined options for PyGRANSO algorithm

In [4]:
opts = Options()
opts.QPsolver = 'osqp' 
opts.x0 = torch.ones((n,1)).to(device=device, dtype=torch.double)
opts.print_level = 1
opts.print_frequency = 10

## Run main algorithm

In [5]:
start = time.time()
soln = pygranso(combinedFunction = comb_fn, objEvalFunction = obj_eval_fn,var_dim_map = var_in, torch_device = device, user_opts = opts)
end = time.time()
print("Total Wall Time: {}s".format(end - start))



╔═════ QP SOLVER NOTICE ════════════════════════════════════════════════════════════════════════╗
║  PyGRANSO requires a quadratic program (QP) solver that has a quadprog-compatible interface,  ║
║  the default is osqp. Users may provide their own wrapper for the QP solver.                  ║
║  To disable this notice, set opts.quadprog_info_msg = False                                   ║
╚═══════════════════════════════════════════════════════════════════════════════════════════════╝
══════════════════════════════════════════════════════════════════════════════════════════════╗
PyGRANSO: Python numerical package using GRadient-based Algorithm for Non-Smooth Optimization ║ 
Version 1.1.0                                                                                 ║ 
MIT License Copyright (c) 2021 SUN Group @ UMN                                                ║ 
══════════════════════════════════════════════════════════════════════════════════════════════╣
Problem specifications:  